In [1]:
import pandas as pd
import geopandas as gpd
# from tabula import read_pdf

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
## Malha de setores do BR
# https://www.ibge.gov.br/geociencias/downloads-geociencias.html?caminho=organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/2021/Malha_de_setores_(shp)_Brasil

In [3]:
## Download das tabelas e documentacao
# https://www.ibge.gov.br/estatisticas/sociais/populacao/9662-censo-demografico-2010.html?edicao=10410

# Selecao das variáveis

## Habitação einfraestrutura

* Porcentagem de domicílios do tipo cômodo .
* Porcentagem de domicílios sem coleta de lixo . 
* Porcentagem de domicílios sem ligação à rede de abastecimento de água .
* Porcentagem de domicílios sem banheiros ou sanitários . 
* Porcentagem de domicílios sem ligação à rede de esgoto ou fossa séptica . 
* Porcentagem de domicílios – outra forma de posse da moradia .
* Número de banheiros por habitante .

## Renda e escolaridade do responsável pelo domicílio

* Porcentagem de responsáveis por domicílio não alfabetizados . 
* Porcentagem de responsáveis por domicílio com menos de 30 anos não alfabetizados . 
* Porcentagem de responsáveis por domicílio com renda de até 3 salários mínimos . 
* Porcentagem de responsáveis por domicílio com menos de 8 anos de estudo . 
* Anos médios de estudo do responsável pelo domicílio . 
* Renda média do responsável pelo domicílio . 

## Aspectos demográficos

* Número de domicílios particulares permanentes no setor censitário . 
* Número de domicílios improvisados no setor censitário . 
* Número de pessoas residentes no setor censitário . 
* Porcentagem de responsáveis por domicílios com menos de 30 anos .
* Número médio de pessoas por domicílio . 



In [4]:
UFS = [
    'AC',
    'AL',
    'AM',
    'AP',
    'BA',
    'CE',
    'DF',
    'ES',
    'GO',
    'MA',
    'MG',
    'MS',
    'MT',
    'PA',
    'PB',
    'PE',
    'PI',
    'PR',
    'RJ',
    'RN',
    'RO',
    'RR',
    'RS',
    'SC',
    'SE',
    'SP1',
    'SP2',
    'TO'
]

In [5]:
## Hash para montar a base de dados
# {'file1': ['var1', 'var2', ..., 'var n'], ...,'file n': ['var1', 'var2', ..., 'var n']}

model_vars = {
    'Basico': {
        # 'Cod_setor': 'cod_setor',
        # 'Cod_Grandes Regiões': 'cod_grande_regiao',
        # 'Nome_Grande_Regiao': 'Nome GRande Região',
        'Cod_UF': 'uf',
        'Nome_da_UF': 'nome_uf',
        'Cod_meso': 'meso_regiao',
        'Cod_RM': 'rm',
        'Nome_da_RM': 'nome_rm',
        # 'Cod_Grandes Regiões': ''
        'Cod_municipio': 'cod_municipio',
        'Nome_do_municipio': 'nome_municipio',
        'Situacao': 'situacao_setor',
        'Tipo_do_setor': 'tipo_setor',
        'Var01': 'numero_domicilios',
        'Var12': 'numero_moradores',
        'Var03': 'rendimento_mensal_responsavel',
    },
    'Domicilio': {
        'V0008': 'domicilios_tipo_comodo',
        'V0048': 'domicilios_com_lixo_coletado',
        'V0123': 'domicilios_com_abastecimento_de_agua',
        'V0134': 'domicilios_com_banheiro',
        'V0030': 'domicilios_com_rede_de_esgoto',
        'V0031': 'domicilios_com_fossa',
        'V0118': 'domicilios_cedidos_de_outra_forma',
        'V0038': 'domicilios_particulares_permanentes_com_1_banheiro',
        'V0039': 'domicilios_particulares_permanentes_com_2_banheiros',
        'V0040': 'domicilios_particulares_permanentes_com_3_banheiros',
        'V0041': 'domicilios_particulares_permanentes_com_4_banheiros',
        'V0042': 'domicilios_particulares_permanentes_com_5_banheiros',
        'V0043': 'domicilios_particulares_permanentes_com_6_banheiros',
        'V0044': 'domicilios_particulares_permanentes_com_7_banheiros',
        'V0045': 'domicilios_particulares_permanentes_com_8_banheiros',
        'V0046': 'domicilios_particulares_permanentes_com_mais_de_9_banheiros',
        'V0004': 'domicilios_particulares_improvisados',
    },
    'Responsavel1': {
        'V0509': 'pessoas_responsaveis_nao_alfabetizadas',
        'V0510': 'pessoas_responsaveis_nao_alfabetizadas_de_10_a_14_anos',
        'V0511': 'pessoas_responsaveis_nao_alfabetizadas_de_15_a_19_anos',
        'V0512': 'pessoas_responsaveis_nao_alfabetizadas_de_20_a_24_anos',
        'V0513': 'pessoas_responsaveis_nao_alfabetizadas_de_25_a_29_anos',
        'V0602': 'responsaveis_com_renda_ate_meio_salario_minimo',
        'V0603': 'responsaveis_com_renda_de_meio_a_1_salario_minimo',
        'V0604': 'responsaveis_com_renda_de_1_a_2_salarios_minimos',
        'V0605': 'responsaveis_com_renda_de_2_a_3_salarios_minimos',
        'V0581': 'responsaveis_com_menos_1_ano_de_estudo',
        'V0582': 'responsaveis_com_menos_2_ano_de_estudo',
        'V0583': 'responsaveis_com_menos_3_ano_de_estudo',
        'V0584': 'responsaveis_com_menos_4_ano_de_estudo',
        'V0585': 'responsaveis_com_menos_5_ano_de_estudo',
        'V0586': 'responsaveis_com_menos_6_ano_de_estudo',
        'V0587': 'responsaveis_com_menos_7_ano_de_estudo',
        'V0402': 'pessoas_responsaveis',
        'V0403': 'pessoas_responsaveis_com_10_anos',
        'V0404': 'pessoas_responsaveis_com_11_anos',
        'V0405': 'pessoas_responsaveis_com_12_anos',
        'V0406': 'pessoas_responsaveis_com_13_anos',
        'V0407': 'pessoas_responsaveis_com_14_anos',
        'V0408': 'pessoas_responsaveis_com_15_anos',
        'V0409': 'pessoas_responsaveis_com_16_anos',
        'V0410': 'pessoas_responsaveis_com_17_anos',
        'V0411': 'pessoas_responsaveis_com_18_anos',
        'V0412': 'pessoas_responsaveis_com_19_anos',
        'V0413': 'pessoas_responsaveis_com_20_anos',
        'V0414': 'pessoas_responsaveis_com_21_anos',
        'V0415': 'pessoas_responsaveis_com_22_anos',
        'V0416': 'pessoas_responsaveis_com_23_anos',
        'V0417': 'pessoas_responsaveis_com_24_anos',
        'V0418': 'pessoas_responsaveis_com_25_anos',
        'V0419': 'pessoas_responsaveis_com_26_anos',
        'V0420': 'pessoas_responsaveis_com_27_anos',
        'V0421': 'pessoas_responsaveis_com_28_anos',
        'V0422': 'pessoas_responsaveis_com_29_anos',
    },
    "Responsavel4": {
        'V1115': 'responsaveis_totais_anos_de_estudo',

    }
}


In [6]:
dfs = []
for uf in UFS:
    print(f'Processando {uf}')
    dfs_uf = []
    for file in model_vars:
        file_path = f'data/censo_2000/csvs/{file}_{uf}.XLS'
        fields = list(model_vars[file].keys())
        fields.append('Cod_setor')
        try:
            df = pd.read_excel(file_path, usecols=fields, dtype="string").set_index('Cod_setor')
        except:
            try:
                df = pd.read_excel(file_path, usecols=fields, dtype="string").set_index('Cod_setor')
            except:
                df = pd.read_excel(file_path, usecols=fields, dtype="string").set_index('Cod_setor')
        df.rename(columns=model_vars[file], inplace=True)
        dfs_uf.append(df)
        # break
    df = pd.concat(dfs_uf, axis=1)
    df.loc[:, 'nome_UF'] = uf
    dfs.append(df)

Processando AC
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Processando AL
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Processando AM
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Processando AP
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is 

In [15]:
df = pd.concat(dfs)

In [16]:
df

,uf,nome_uf,meso_regiao,rm,nome_rm,cod_municipio,nome_municipio,situacao_setor,tipo_setor,numero_domicilios,...,responsaveis_com_menos_4_ano_de_estudo,responsaveis_com_menos_5_ano_de_estudo,responsaveis_com_menos_6_ano_de_estudo,responsaveis_com_menos_7_ano_de_estudo,responsaveis_com_renda_ate_meio_salario_minimo,responsaveis_com_renda_de_meio_a_1_salario_minimo,responsaveis_com_renda_de_1_a_2_salarios_minimos,responsaveis_com_renda_de_2_a_3_salarios_minimos,responsaveis_totais_anos_de_estudo,nome_UF
Cod_setor,,,,,,,,,,,,,,,,,,,,,
120001305000001,12,Acre,1202,00,Não pertence,1200013,Acrelândia,1,0,251,...,42,16,7,7,0,53,57,28,1092,AC
120001305000002,12,Acre,1202,00,Não pertence,1200013,Acrelândia,1,0,311,...,46,27,7,11,6,89,79,25,1023,AC
120001305000003,12,Acre,1202,00,Não pertence,1200013,Acrelândia,1,0,302,...,47,22,4,5,7,87,79,23,1024,AC
120001305000004,12,Acre,1202,00,Não pertence,1200013,Acrelândia,5,0,49,...,8,4,1,3,1,10,24,2,168,AC
120001305000005,12,Acre,1202,00,Não pertence,1200013,Acrelândia,8,0,98,...,15,4,6,1,2,21,31,6,213,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172210705000011,17,Tocantins,1701,00,Não pertence,1722107,Xambioá,5,0,28,...,2,1,1,0,1,13,10,0,36,TO
172210705000012,17,Tocantins,1701,00,Não pertence,1722107,Xambioá,8,0,119,...,11,6,1,5,9,50,29,14,<NA>,TO
172210705000013,17,Tocantins,1701,00,Não pertence,1722107,Xambioá,8,0,155,...,20,10,4,4,9,52,57,15,338,TO


In [10]:
# Adicionar MacroRegioes
grandes_regioes = pd.read_csv('data/grandes_regioes.csv')

In [17]:
df = df.reset_index().merge(grandes_regioes, on='nome_UF', how='left')

In [19]:
df.set_index('Cod_setor', inplace=True)

In [20]:
df

,uf,nome_uf,meso_regiao,rm,nome_rm,cod_municipio,nome_municipio,situacao_setor,tipo_setor,numero_domicilios,...,responsaveis_com_menos_5_ano_de_estudo,responsaveis_com_menos_6_ano_de_estudo,responsaveis_com_menos_7_ano_de_estudo,responsaveis_com_renda_ate_meio_salario_minimo,responsaveis_com_renda_de_meio_a_1_salario_minimo,responsaveis_com_renda_de_1_a_2_salarios_minimos,responsaveis_com_renda_de_2_a_3_salarios_minimos,responsaveis_totais_anos_de_estudo,nome_UF,cod_grande_regiao
Cod_setor,,,,,,,,,,,,,,,,,,,,,
120001305000001,12,Acre,1202,00,Não pertence,1200013,Acrelândia,1,0,251,...,16,7,7,0,53,57,28,1092,AC,1.0
120001305000002,12,Acre,1202,00,Não pertence,1200013,Acrelândia,1,0,311,...,27,7,11,6,89,79,25,1023,AC,1.0
120001305000003,12,Acre,1202,00,Não pertence,1200013,Acrelândia,1,0,302,...,22,4,5,7,87,79,23,1024,AC,1.0
120001305000004,12,Acre,1202,00,Não pertence,1200013,Acrelândia,5,0,49,...,4,1,3,1,10,24,2,168,AC,1.0
120001305000005,12,Acre,1202,00,Não pertence,1200013,Acrelândia,8,0,98,...,4,6,1,2,21,31,6,213,AC,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172210705000011,17,Tocantins,1701,00,Não pertence,1722107,Xambioá,5,0,28,...,1,1,0,1,13,10,0,36,TO,1.0
172210705000012,17,Tocantins,1701,00,Não pertence,1722107,Xambioá,8,0,119,...,6,1,5,9,50,29,14,<NA>,TO,1.0
172210705000013,17,Tocantins,1701,00,Não pertence,1722107,Xambioá,8,0,155,...,10,4,4,9,52,57,15,338,TO,1.0


In [21]:
CAMPOS_MODELO = [
    'cod_grande_regiao', #
    'uf', #
    'nome_UF', #
    'rm', #
    'nome_rm', #
    'meso_regiao', #
    'cod_municipio', #
    'nome_municipio', #
    'situacao_setor',#
    'tipo_setor',#
    'numero_de_domicilios',#
    'numero_moradores',#
    # 'area_do_setor',
    'porcentagem_domicilios_tipo_comodo', #
    'porcentagem_domicilios_sem_coleta_de_lixo', #
    'porcentagem_domicilios_sem_ligacao_rede_de_agua', #
    'porcentagem_domicilios_sem_banheiros', #
    'porcentagem_domicilios_sem_esgoto_ou_fossa', #
    'porcentagem_domicilios_cedidos_outra_forma', #
    # 'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante', #
    'porcentagem_responsaveis_nao_alfabetizado', #
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos', #
    'porcentagem_de_responsaveis_com_menos_30_anos', #
    'porcentagem_responsaveis_renda_ate_3_salarios', #
    'porcentagem_responsaveis_menos_8_anos_de_estudo', #
    'anos_medios_estudo_responsavel', #
    'renda_media_responsavel', #
    # 'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados', #
    # 'numero_de_pessoas_residentes',
    'numero_medio_pessoas', #
    # 'densidade_populacional',
    # 'densidade_domicilios'
]

In [22]:
df.loc[:, 'numero_de_domicilios'] = pd.to_numeric(df.loc[:, 'numero_domicilios'], errors='coerce')
# df.loc[:, 'numero_de_domicilios'].fillna(0., inplace=True)
df.loc[:, 'numero_moradores'] = pd.to_numeric(df.loc[:, 'numero_moradores'], errors='coerce')
# df.loc[:, 'numero_de_domicilios'].fillna(0., inplace=True)

In [23]:
# df.loc[:, CAMPOS_MODELO]

In [24]:
df.loc[:, 'porcentagem_domicilios_tipo_comodo'] = pd.to_numeric(df.loc[:, 'domicilios_tipo_comodo'], errors='coerce') / df.numero_de_domicilios

In [25]:
df.loc[:, 'porcentagem_domicilios_sem_coleta_de_lixo'] = (df.numero_de_domicilios - pd.to_numeric(df.loc[:, 'domicilios_com_lixo_coletado'], errors='coerce')) / df.numero_de_domicilios

In [26]:
df.loc[:, 'porcentagem_domicilios_sem_ligacao_rede_de_agua'] = (df.numero_de_domicilios - pd.to_numeric(df.loc[:, 'domicilios_com_abastecimento_de_agua'], errors='coerce')) / df.numero_de_domicilios

In [27]:
df.loc[:, 'porcentagem_domicilios_sem_banheiros'] =  (df.numero_de_domicilios - pd.to_numeric(df.loc[:, 'domicilios_com_banheiro'], errors='coerce')) / df.numero_de_domicilios

In [28]:
# 'porcentagem_domicilios_sem_esgoto_ou_fossa'
df.loc[:, 'porcentagem_domicilios_sem_esgoto_ou_fossa'] = (df.numero_de_domicilios - pd.to_numeric(df.loc[:, 'domicilios_com_rede_de_esgoto'], errors='coerce') - pd.to_numeric(df.loc[:, 'domicilios_com_fossa'], errors='coerce')) / df.numero_de_domicilios


In [29]:
# porcentagem_domicilios_cedidos_outra_forma
df.loc[:, 'porcentagem_domicilios_cedidos_outra_forma'] = pd.to_numeric(df.loc[:, 'domicilios_cedidos_de_outra_forma'], errors='coerce') / df.numero_de_domicilios

In [30]:
banheiros = [
    'domicilios_particulares_permanentes_com_1_banheiro',
    'domicilios_particulares_permanentes_com_2_banheiros',
    'domicilios_particulares_permanentes_com_3_banheiros',
    'domicilios_particulares_permanentes_com_4_banheiros',
    'domicilios_particulares_permanentes_com_5_banheiros',
    'domicilios_particulares_permanentes_com_6_banheiros',
    'domicilios_particulares_permanentes_com_7_banheiros',
    'domicilios_particulares_permanentes_com_8_banheiros',
    'domicilios_particulares_permanentes_com_mais_de_9_banheiros'
]

df.loc[:, 'banheiros'] = 0

for k,v in list(zip(range(9), banheiros)):
    df.loc[:, 'banheiros'] += pd.to_numeric(df.loc[:, v], errors='coerce') * (k+1)

# numero_de_banheiros_por_habitante
df.loc[:, 'numero_de_banheiros_por_habitante'] = df.banheiros / df.numero_de_domicilios

In [31]:
# porcentagem_responsaveis_nao_alfabetizado
df.loc[:, 'porcentagem_responsaveis_nao_alfabetizado'] =  pd.to_numeric(df.loc[:, 'pessoas_responsaveis_nao_alfabetizadas'], errors='coerce') / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [32]:
# porcentagem_de_responsaveis_com_menos_30_anos
        
responsaveis_menos_30 = [
        'pessoas_responsaveis_com_10_anos',
        'pessoas_responsaveis_com_11_anos',
        'pessoas_responsaveis_com_12_anos',
        'pessoas_responsaveis_com_13_anos',
        'pessoas_responsaveis_com_14_anos',
        'pessoas_responsaveis_com_15_anos',
        'pessoas_responsaveis_com_16_anos',
        'pessoas_responsaveis_com_17_anos',
        'pessoas_responsaveis_com_18_anos',
        'pessoas_responsaveis_com_19_anos',
        'pessoas_responsaveis_com_20_anos',
        'pessoas_responsaveis_com_21_anos',
        'pessoas_responsaveis_com_22_anos',
        'pessoas_responsaveis_com_23_anos',
        'pessoas_responsaveis_com_24_anos',
        'pessoas_responsaveis_com_25_anos',
        'pessoas_responsaveis_com_26_anos',
        'pessoas_responsaveis_com_27_anos',
        'pessoas_responsaveis_com_28_anos',
        'pessoas_responsaveis_com_29_anos',
]

df.loc[:, 'responsaveis_menos_30'] = 0

for v in responsaveis_menos_30:
    df.loc[:, 'responsaveis_menos_30'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_de_responsaveis_com_menos_30_anos'] = df.responsaveis_menos_30 / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [33]:
# porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos
nao_alfabetizados_menos_30 = [
    'pessoas_responsaveis_nao_alfabetizadas_de_10_a_14_anos',
    'pessoas_responsaveis_nao_alfabetizadas_de_15_a_19_anos',
    'pessoas_responsaveis_nao_alfabetizadas_de_20_a_24_anos',
    'pessoas_responsaveis_nao_alfabetizadas_de_25_a_29_anos'
]

df.loc[:, 'nao_alfabetizados_menos_30'] = 0

for v in nao_alfabetizados_menos_30:
    df.loc[:, 'nao_alfabetizados_menos_30'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos'] = df.loc[:, 'nao_alfabetizados_menos_30']  /pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [34]:
# porcentagem_responsaveis_renda_ate_3_salarios
responsaveis_renda_ate_3_salarios = [
        'responsaveis_com_renda_ate_meio_salario_minimo',
        'responsaveis_com_renda_de_meio_a_1_salario_minimo',
        'responsaveis_com_renda_de_1_a_2_salarios_minimos',
        'responsaveis_com_renda_de_2_a_3_salarios_minimos'
]

df.loc[:, 'responsaveis_renda_ate_3_salarios'] = 0

for v in responsaveis_renda_ate_3_salarios:
    df.loc[:, 'responsaveis_renda_ate_3_salarios'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_responsaveis_renda_ate_3_salarios'] = (df.loc[:, 'responsaveis_renda_ate_3_salarios']) / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [35]:
reponsaveis_menos_8_anos_estudo = [
    'responsaveis_com_menos_1_ano_de_estudo',
    'responsaveis_com_menos_2_ano_de_estudo',
    'responsaveis_com_menos_3_ano_de_estudo',
    'responsaveis_com_menos_4_ano_de_estudo',
    'responsaveis_com_menos_5_ano_de_estudo',
    'responsaveis_com_menos_6_ano_de_estudo',
    'responsaveis_com_menos_7_ano_de_estudo',
]

df.loc[:, 'reponsaveis_menos_8_anos_estudo'] = 0

for v in reponsaveis_menos_8_anos_estudo:
    df.loc[:, 'reponsaveis_menos_8_anos_estudo'] += pd.to_numeric(df.loc[:, v], errors='coerce')

df.loc[:, 'porcentagem_responsaveis_menos_8_anos_de_estudo'] = (df.loc[:, 'reponsaveis_menos_8_anos_estudo']) / pd.to_numeric(df.loc[:, 'pessoas_responsaveis'], errors='coerce')

In [36]:
df.loc[:, 'anos_medios_estudo_responsavel']  = pd.to_numeric(df.loc[:, 'responsaveis_totais_anos_de_estudo'], errors='coerce')

In [37]:
df.loc[:, 'renda_media_responsavel']  = pd.to_numeric(df.loc[:, 'rendimento_mensal_responsavel'], errors='coerce')

In [38]:
# numero_de_domicilios_improvisados
# total_domicilios_improvisados
df.loc[:, 'numero_de_domicilios_improvisados'] = pd.to_numeric(df.loc[:, 'domicilios_particulares_improvisados'], errors='coerce')

In [39]:
# numero_medio_pessoas'
df.loc[:, 'numero_medio_pessoas'] = pd.to_numeric(df.loc[:, 'numero_moradores'], errors='coerce') / pd.to_numeric(df.loc[:, 'numero_de_domicilios'], errors='coerce')

In [40]:
# df.loc[:, 'densidade_populacional'] = pd.to_numeric(df.loc[:, 'numero_moradores'], errors='coerce') / df.loc[:, 'area_do_setor']

In [41]:
# df.loc[:, 'densidade_domicilios'] = pd.to_numeric(df.loc[:, 'numero_de_domicilios'], errors='coerce') / df.loc[:, 'area_do_setor']

In [42]:
df.loc[:, CAMPOS_MODELO]

,cod_grande_regiao,uf,nome_UF,rm,nome_rm,meso_regiao,cod_municipio,nome_municipio,situacao_setor,tipo_setor,...,numero_de_banheiros_por_habitante,porcentagem_responsaveis_nao_alfabetizado,porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos,porcentagem_de_responsaveis_com_menos_30_anos,porcentagem_responsaveis_renda_ate_3_salarios,porcentagem_responsaveis_menos_8_anos_de_estudo,anos_medios_estudo_responsavel,renda_media_responsavel,numero_de_domicilios_improvisados,numero_medio_pessoas
Cod_setor,,,,,,,,,,,,,,,,,,,,,
120001305000001,1.0,12,AC,00,Não pertence,1202,1200013,Acrelândia,1,0,...,0.585657,0.852590,0.175299,0.183267,0.549801,0.442231,1092.0,597.119522,0,4.147410
120001305000002,1.0,12,AC,00,Não pertence,1202,1200013,Acrelândia,1,0,...,0.196141,0.688103,0.289389,0.334405,0.639871,0.475884,1023.0,286.742765,0,3.832797
120001305000003,1.0,12,AC,00,Não pertence,1202,1200013,Acrelândia,1,0,...,0.380795,0.731788,0.201987,0.241722,0.649007,0.500000,1024.0,418.423841,2,4.158940
120001305000004,1.0,12,AC,00,Não pertence,1202,1200013,Acrelândia,5,0,...,0.224490,0.795918,0.142857,0.142857,0.755102,0.510204,168.0,383.122449,0,4.183673
120001305000005,1.0,12,AC,00,Não pertence,1202,1200013,Acrelândia,8,0,...,0.112245,0.581633,0.091837,0.153061,0.612245,0.551020,213.0,540.775510,1,4.265306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172210705000011,1.0,17,TO,00,Não pertence,1701,1722107,Xambioá,5,0,...,0.357143,0.535714,0.071429,0.107143,0.857143,0.464286,36.0,185.178571,0,4.107143
172210705000012,1.0,17,TO,00,Não pertence,1701,1722107,Xambioá,8,0,...,0.184874,0.663866,0.134454,0.210084,0.857143,0.731092,NaN,226.327731,10,4.605042
172210705000013,1.0,17,TO,00,Não pertence,1701,1722107,Xambioá,8,0,...,0.277419,0.612903,0.174194,0.238710,0.858065,0.638710,338.0,284.432258,1,3.967742


In [43]:
df.loc[:, CAMPOS_MODELO].to_csv('resultados/modelo-2000.csv.zip', compression='zip')

In [44]:
df_ = pd.read_csv('resultados/modelo-2000.csv.zip', compression='zip')

In [45]:
df_.columns

Index(['Cod_setor', 'cod_grande_regiao', 'uf', 'nome_UF', 'rm', 'nome_rm',
       'meso_regiao', 'cod_municipio', 'nome_municipio', 'situacao_setor',
       'tipo_setor', 'numero_de_domicilios', 'numero_moradores',
       'porcentagem_domicilios_tipo_comodo',
       'porcentagem_domicilios_sem_coleta_de_lixo',
       'porcentagem_domicilios_sem_ligacao_rede_de_agua',
       'porcentagem_domicilios_sem_banheiros',
       'porcentagem_domicilios_sem_esgoto_ou_fossa',
       'porcentagem_domicilios_cedidos_outra_forma',
       'numero_de_banheiros_por_habitante',
       'porcentagem_responsaveis_nao_alfabetizado',
       'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
       'porcentagem_de_responsaveis_com_menos_30_anos',
       'porcentagem_responsaveis_renda_ate_3_salarios',
       'porcentagem_responsaveis_menos_8_anos_de_estudo',
       'anos_medios_estudo_responsavel', 'renda_media_responsavel',
       'numero_de_domicilios_improvisados', 'numero_medio_pessoas'],
 